# Homework \#6
Due __Friday, August 6__ @ 11:59pm

## Submission requirements
Upload a **single PDF file** of your IJulia notebook for this entire assigment. Clearly denote which question each section of your PDF corresponds to.

## Problem 1 -- Bakeshop

A bakery must complete three prep tasks before assembling a cake. These tasks can be done in any order.

Task 1 (making the cake) has three steps that have to be done in any order:
* Mix ingredients
* Bake cake
* Cool cake

Task 2 (making the frosting) has two steps that can be done in any order:
* Stir frosting
* Taste test frosting

Task 3 (make decorations) has three steps that can be done in any order:
* Make fondant
* Make chocolate sprinkles
* Make rainbow sprinkles

There are four cake assembly stations. The tasks can be done at any of the four stations.

Our goal is to minimize the total completion time for assembling cakes.  Key issues to deal with are:
- A station cannot handle multiple jobs simultaneously (e.g., cannot mix ingredients and stir frosting at the same time)
- Each job must be completed at a single station (e.g., cannot start making fondant at one station and finish it at the other). 
- Tasks can be split across stations (e.g., chocolate sprinkles can be made at one station while rainbow sprinkles are made at another station).
- All jobs must be completed in order to finish a cake assembly.

Jobs are defined by pairs (task,step) that have to get done. Because of different levels of expertise of the bakers, it takes different amounts of time to complete each step at each of the four assembly stations. Processing times (in minutes) for each job at each station are given below:

|Station|1|2|3|4|
|:---|:-|:-|:-|:-|
|(Task1,Step1)|17|18|12|16|
|(Task1,Step2)|20|18|17|14|
|(Task1,Step3)|17|21|9|21|
|(Task2,Step1)|13|18|21|12|
|(Task2,Step2)|19|21|17|15|
|(Task3,Step1)|11|13|9|14|
|(Task3,Step2)|13|9|9|12|
|(Task3,Step3)|9|13|17|17|

It takes 10 minutes to clean and prep an assembly station in order to use it. 

Each assembly station needs to be used for other tasks during the day so the stations have the following limited total availability (in minutes):

|Station| Time available (min)|
|:------|:-----|
|1|41|
|2|45|
|3|50|
|4|46|



Formulate an integer program to help the bakeshop assign (task,step) pairs to assembly stations in order to minimize the total time to complete a cake assembly. (Hint: model the problem as a generalized assignment problem.)

In [3]:
using NamedArrays, JuMP, Gurobi

tasks = [:1_1, :1_2, :1_3, :2_1, :2_2, :3_1, :3_2, :3_3]
stations = [:1, :2, :3, :4]

b = [41 45 50 46] # capacity of each station

# cost matrix
c = Matrix([
17 18 12 16
20 18 17 14
17 21  9 21
13 18 21 12
19 21 17 15
11 13  9 14
13  9  9 12
9  13 17 17])

# fixed cost
h = [10 10 10 10]

m = length(stations) # number of stations = 4
n = length(tasks) # number of tasks = 8

8

In [4]:
mod = Model(Gurobi.Optimizer)
set_optimizer_attribute(mod, "OutputFlag", 0)

# x[i,j] = 1 : if task j is assigned to station i
# z[i] = 1 : if station i is used

# binary variables that assign tasks to stations
@variable(mod, x[1:m, 1:n], Bin)

# binary variables that tell us which stations to use
@variable(mod, z[1:n], Bin)

# objective is to minimize the total assembly time
# m = 4, n = 8
@objective(mod, Min, sum(c[j,i] * x[i,j] for i in 1:m for j in 1:n) + sum(h[i] * z[i] for i in 1:m))
#@objective(mod, Min, sum(c * x) + sum(h[i] * z[i] for i in 1:m))

# Each job must be completed at a single station
@constraint(mod, jobassign[i in 1:n], sum(x[j,i] for j in 1:m) == 1)

# station capacity
@constraint(mod, capacity[i in 1:m], sum(c[j,i] * x[i,j] for j in 1:n) <= b[i])

# A station cannot handle multiple jobs simulataneously
@constraint(mod, logic[i in 1:m], sum(c[j,i] * x[i,j] for j in 1:n) <= b[i] * z[i])

optimize!(mod)

sol = NamedArray(zeros(m,n),(stations,tasks),("stations","tasks"))

for i in 1:m
    for j in 1:n
        sol[i,j] = value(x[i,j])
    end
end
println(sol)
println()
println("Total Assembly time: ", value.(sum(c[j,i] * x[i,j] for i in 1:m for j in 1:n)))
println("Total Fixed Cost time: ", value.(sum(h[i] * z[i] for i in 1:m)))
println("Final Minimum Total Time: ", objective_value(mod))

Academic license - for non-commercial use only - expires 2022-07-06
4×8 Named Matrix{Float64}
stations ╲ tasks │   11    12    13    21    22    31    32    33
─────────────────┼───────────────────────────────────────────────
1                │ -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   1.0
2                │ -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   0.0  -0.0
3                │  1.0  -0.0   1.0  -0.0  -0.0   1.0   1.0  -0.0
4                │ -0.0   1.0  -0.0   1.0   1.0  -0.0  -0.0  -0.0

Total Assembly time: 89.0
Total Fixed Cost time: 30.0
Final Minimum Total Time: 119.0
